In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

In [3]:
full = pd.read_csv('../../../full.csv')

C:\Users\weisj\anaconda3\envs\dsi\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (351) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
full.drop(columns  = ['Unnamed: 0', 'Unnamed: 0_x'], inplace = True)
full.columns = full.columns.str.lower()


In [ ]:
full.to_csv('full.csv', index =False)